In [ ]:
import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor
import os
import arcpy
arcpy.env.overwriteOutput = True

In [ ]:
"""
    Variables
"""

xlsx_informalidades = r"C:\Users\rodian.saby\OneDrive\Documentos\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\zApoyo_A_Equipos\zApoyoEquipoJuridico\5.EspacializacionInformalidades\zInsumos\RPT_31084_INFORMALIDADES.xlsx"

columnas_salida = ['CODIGO',
    'CODIGO_ANTERIOR',
    'SHAPE',
    'ficha',
    'npn',
    'sector',
    'municipio']

renombre_columnas = {'CODIGO':'codigo',
    'CODIGO_ANTERIOR':'codigo_anterior',
}


In [ ]:
HOJA = 'FICHAS'

df_informalidades = pd.read_excel(xlsx_informalidades, sheet_name = HOJA)

In [ ]:

# ** Generación de nueva columna
df_informalidades['npn_parametrizado'] = ''

<h2>Parametrización de registros por medio de apply</h2>

In [ ]:

# ** Define una función que realice el slicing a un string, para eliminar el 1 al final de la cadena de caracteres
def eliminacion_mejora(text):
    # ? Función que me verifica si el tipo de dato parametrizado es el que ingresa a la función
    if isinstance(text, str) and text[21] == '5':
        cadena = text[:21] + '000000000'
        return cadena 
    else:
        return None  # Devuelve None si el valor no es un string


# ** Aplica la función a cada elemento de la columna npn y asigna el resultado a npn_parametrizado
df_informalidades['npn_parametrizado'] = df_informalidades['npn'].apply(eliminacion_mejora)

<h2>Espacialización Terrenos</h2>

In [ ]:
RUTA_BDCONSOLIDADA = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E1_Alistamiento_Diagnostico\3_Disposicion\1.BD_Consolidada\BD_Consolidada_Lote4.gdb"
ruta_shp = r"C:\Users\rodian.saby\OneDrive\Documentos\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\zApoyo_A_Equipos\zApoyoEquipoJuridico\5.EspacializacionInformalidades\zShp"

CAPA_TERRENOS_R = 'Analitica_UT_Consolidada\R_TERRENO'
CAPA_TERRENOS_U = 'Analitica_UT_Consolidada\\U_TERRENO'
SHP = 'Posibles_Predios_Informales.shp'

capa_terrenos_rurales = os.path.join(RUTA_BDCONSOLIDADA, CAPA_TERRENOS_R)
capa_terrenos_urbanos = os.path.join(RUTA_BDCONSOLIDADA, CAPA_TERRENOS_U)

# ** Terrenos Rurales
df_terreno_rural = pd.DataFrame.spatial.from_featureclass(capa_terrenos_rurales)
df_terreno_rural.head()
# ** Terrenos Urbanos
df_terreno_urbano = pd.DataFrame.spatial.from_featureclass(capa_terrenos_urbanos)

# ** Selección de columnas
columnas_selecciones = ['CODIGO',
    'CODIGO_ANTERIOR',
    'SHAPE']

# ** DF Terrenos Rurales, Urbanos
df_terreno_rural_reducido = df_terreno_rural[columnas_selecciones]
df_terreno_urbano_reducido = df_terreno_urbano[columnas_selecciones]

# ** Concatenación de dataframes
df_terrenos = pd.concat([df_terreno_rural_reducido, df_terreno_urbano_reducido])

In [ ]:

# ** Terrenos Gestionados SIG Espacializados
df_terrenos_posible_informalidad = pd.merge(df_terrenos, df_informalidades, left_on='CODIGO', right_on='npn_parametrizado', how='inner')

<h2>Estandarización de DF</h2>

In [ ]:
df_posible_informalidad_filtrado = df_terrenos_posible_informalidad[columnas_salida]

df_posible_informalidad_estandarizado = df_posible_informalidad_filtrado.rename(columns=renombre_columnas)

In [ ]:

# ** Exportación a BD
NOMBRE_CAPA_SALIDA = f"Analitica_UT_Consolidada\\{'posibles_predios_informales'.upper()}"
ruta_salida_terrenos_contro_cambios = os.path.join(RUTA_BDCONSOLIDADA, NOMBRE_CAPA_SALIDA)

df_posible_informalidad_estandarizado.spatial.to_featureclass(location=ruta_salida_terrenos_contro_cambios)
print(f"Se exporta la capa {NOMBRE_CAPA_SALIDA}")

# ** Exportación SHP
ruta_salida_shp = os.path.join(ruta_shp, SHP)
df_posible_informalidad_estandarizado.spatial.to_featureclass(location=ruta_salida_shp)
print(f"Se exporta la capa {SHP}")